# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

- Profile dataset and catalog data quality issues
- Define cleaning rules and business assumptions with a data dictionary
- Standardize schema and infer types for core fields
- Normalize currencies to USD and consolidate compensation components
- Standardize job titles and map to canonical roles
- Clean and geocode locations; extract and validate US states
- Parse and normalize years of experience fields
- Handle missing, invalid, and duplicate records
- Detect and mitigate outliers with defensible rules
- Compute cleaned salary metrics and derived features
- Answer four business questions with validated calculations
- Document assumptions, QA checks, and deliver final summary


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [356]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

df_messy=pd.read_csv('../../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv',delimiter='\t')
print('rows, cols:',df_messy.shape)
print('************************************************')
print(df_messy.dtypes)
print('************************************************')
print('number of NaNs:',df_messy.isna().sum())
print('************************************************')
df_messy.head()

rows, cols: (28062, 18)
************************************************
Timestamp                                                                                                                                                                                                                                object
How old are you?                                                                                                                                                                                                                         object
What industry do you work in?                                                                                                                                                                                                            object
Job title                                                                                                                                                                                                      

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


## Step 2: Data Cleaning


In [357]:

column_map = {
    'Timestamp' : 'time_of_response',
    'How old are you?' : 'age',
    'What industry do you work in?' : 'industry',
    'Job title' : 'job_title',
    'If your job title needs additional context, please clarify here:' : 'extra_job_info',
    'What is your annual salary? (You\'ll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)' : 'annual_sal_full_time',
    'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.' : 'additional_compensation',
    'Please indicate the currency' : 'currency_type',
    'If "Other," please indicate the currency here: ' : 'uncommon_currency_type',
    'If your income needs additional context, please provide it here:' : 'income_context',
    'What country do you work in?' : 'country',
    'If you\'re in the U.S., what state do you work in?' : 'us_state',
    'What city do you work in?' : 'city',
    'How many years of professional work experience do you have overall?' : 'yrs_experience_general',
    'How many years of professional work experience do you have in your field?' : 'yrs_experience_field',
    'What is your highest level of education completed?' : 'highest_lvl_edu',
    'What is your gender?' : 'gender',
    'What is your race? (Choose all that apply.)' : 'race'
}

df_clean = df_messy.rename(columns=column_map)
df_clean.head()

,time_of_response,age,industry,job_title,extra_job_info,annual_sal_full_time,additional_compensation,currency_type,uncommon_currency_type,income_context,country,us_state,city,yrs_experience_general,yrs_experience_field,highest_lvl_edu,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [358]:
america = ['United States', 'United States ', 'USA', 'US', 'U.S.', 'United States of America', 'USA ', 'U.S', 'U.S.A', 'Usa',
           'United states', 'united states', 'U.S.A.', 'United States of America ', 'usa' , 'Us','us']
df_clean['country'] = df_clean['country'].replace(america, 'USA')

print(df_clean['country'].value_counts())

df_clean['country'].isna().sum()
df_clean.head()


country
USA               22897
Canada             1570
UK                  574
United Kingdom      547
Australia           318
                  ...  
LOUTRELAND            1
philippines           1
ff                    1
Myanmar               1
Burma                 1
Name: count, Length: 363, dtype: int64


,time_of_response,age,industry,job_title,extra_job_info,annual_sal_full_time,additional_compensation,currency_type,uncommon_currency_type,income_context,country,us_state,city,yrs_experience_general,yrs_experience_field,highest_lvl_edu,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,USA,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,USA,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,USA,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [359]:
print(df_clean.isna().sum())
#no NaNs in the following cols: 
#time_of_response, age, annual_sal_full_time, currency_type, country, yrs_experience_general, yrs_experience_field
print(df_clean['currency_type'].value_counts())


condition = df_clean['currency_type'] == 'Other'
df_clean.loc[condition, 'currency_type'] = df_clean['uncommon_currency_type']

df_clean['currency_type'].value_counts()

df_clean = df_clean.drop(columns=['uncommon_currency_type'], inplace=False)

df_clean.head()


time_of_response               0
age                            0
industry                      74
job_title                      1
extra_job_info             20800
annual_sal_full_time           0
additional_compensation     7296
currency_type                  0
uncommon_currency_type     27856
income_context             25020
country                        0
us_state                    5023
city                          82
yrs_experience_general         0
yrs_experience_field           0
highest_lvl_edu              222
gender                       171
race                         177
dtype: int64
currency_type
USD        23374
CAD         1673
GBP         1591
EUR          643
AUD/NZD      504
Other        160
CHF           37
SEK           37
JPY           23
ZAR           16
HKD            4
Name: count, dtype: int64


,time_of_response,age,industry,job_title,extra_job_info,annual_sal_full_time,additional_compensation,currency_type,income_context,country,us_state,city,yrs_experience_general,yrs_experience_field,highest_lvl_edu,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,USA,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,USA,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,USA,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [360]:
print(df_clean['job_title'].value_counts())

mask = df_clean['job_title'].str.contains('Software Engineer', na=False)
# Use the mask to filter the DataFrame
swe_rows = df_clean[mask]
print(swe_rows['job_title'].value_counts())
swe_rows

job_title
Software Engineer                          286
Project Manager                            230
Director                                   198
Senior Software Engineer                   196
Program Manager                            152
                                          ... 
Director Accounting                          1
Research Strategy and Portfolio Manager      1
QA Assistant Manager                         1
Bicycle mechanic                             1
Sales rep for fuel, oil and lubes            1
Name: count, Length: 14348, dtype: int64
job_title
Software Engineer                      286
Senior Software Engineer               196
Staff Software Engineer                 36
Software Engineer II                    31
Principal Software Engineer             30
                                      ... 
Software Engineering manager             1
Software Engineering Intern              1
Full Stack Software Engineer II          1
Software Engineer 1                

,time_of_response,age,industry,job_title,extra_job_info,annual_sal_full_time,additional_compensation,currency_type,income_context,country,us_state,city,yrs_experience_general,yrs_experience_field,highest_lvl_edu,gender,race
43,4/27/2021 11:04:04,25-34,Computing or Tech,Principal Software Engineer,NaN,"187,500",5000.0,USD,NaN,USA,Pennsylvania,Pittsburgh,8 - 10 years,5-7 years,College degree,Woman,White
321,4/27/2021 11:08:11,35-44,Computing or Tech,Software Engineer,NaN,"156,000",189000.0,USD,"Salary, stock, bonus",USA,Massachusetts,Metro Boston,21 - 30 years,21 - 30 years,Master's degree,Other or prefer not to answer,Another option not listed here or prefer not t...
389,4/27/2021 11:09:13,35-44,Computing or Tech,Software Engineer,NaN,"112,000",0.0,USD,NaN,USA,Florida,Melbourne,11 - 20 years,8 - 10 years,College degree,Woman,White
455,4/27/2021 11:10:12,25-34,Computing or Tech,Software Engineer,NaN,"80,000",3000.0,USD,NaN,USA,Minnesota,Minneapolis,1 year or less,1 year or less,Some college,Non-binary,White
511,4/27/2021 11:11:06,45-54,Oil and Gas,Senior Software Engineer,NaN,"120,000",20000.0,USD,NaN,USA,Texas,Houston,21 - 30 years,21 - 30 years,Some college,Man,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27745,4/6/2022 13:49:57,25-34,Computing or Tech,Software Engineer III,NaN,125000,10000.0,USD,NaN,USA,Georgia,Atlanta,5-7 years,2 - 4 years,College degree,Man,White
27750,4/6/2022 20:25:25,25-34,Computing or Tech,Software Engineer,MLOps,340000,0.0,USD,Working two remote Jobs right now.,Usa,"Michigan, Texas, Washington",Houston,2 - 4 years,2 - 4 years,Master's degree,Man,"Asian or Asian American, Hispanic, Latino, or ..."
27755,4/8/2022 8:46:10,25-34,Computing or Tech,Software Engineer,NaN,35000,55.0,GBP,NaN,United Kingdom,NaN,London,1 year or less,1 year or less,College degree,Man,White
27804,5/17/2022 0:08:55,35-44,Computing or Tech,Software Engineer,NaN,152000,20000.0,USD,NaN,USA,California,Walnut Creek,11 - 20 years,2 - 4 years,College degree,Woman,White


## Step 3: Business Questions Analysis

Now answer those important business questions!


In [361]:
# Question 1: What is the median salary for Software Engineers in the United States?
swe_rows.dtypes
# 'annual_sal_full_time        object' -> need to convert to num
swe_rows['annual_sal_full_time'] = pd.to_numeric(swe_rows['annual_sal_full_time'], errors='coerce')
swe_rows.dtypes
# 'annual_sal_full_time       float64'

usa_mask = swe_rows['country'] == 'USA'
usa_salaries = swe_rows.loc[usa_mask, 'annual_sal_full_time']
median_swe_sal = usa_salaries.median()
print(f"Median salary for Software Engineers in the US: ${median_swe_sal:,.0f}")

#ANS : Median salary for Software Engineers in the US: $142,000


Median salary for Software Engineers in the US: $142,000


/tmp/ipykernel_47584/1792450171.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [362]:
# Question 2: Which US state has the highest average salary for tech workers?
condition = (df_clean['industry'] == 'Computing or Tech') & (df_clean['country'] == 'USA')
tech_wrk_us = df_clean[condition]
tech_wrk_us['annual_sal_full_time'] = pd.to_numeric(tech_wrk_us['annual_sal_full_time'], errors='coerce')

tech_wrk_us['us_state'].value_counts()

tech_wrk_us_avg = tech_wrk_us.groupby('us_state')['annual_sal_full_time'].mean().reset_index()
sorted_average_df = tech_wrk_us_avg.sort_values(by='annual_sal_full_time', ascending=False)
print('US state with the highest avg salary for tech workers:',sorted_average_df.iloc[0, 0])

#ANS : US state with the highest avg salary for tech workers: Florida


US state with the highest avg salary for tech workers: Florida


/tmp/ipykernel_47584/105405403.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [363]:
# Question 3: How much does salary increase on average for each year of experience in tech?
condition = (df_clean['industry'] == 'Computing or Tech')
df_yrs = df_clean[condition]
df_yrs['annual_sal_full_time'] = pd.to_numeric(df_yrs['annual_sal_full_time'], errors='coerce')

df_yrs_avg = df_yrs.groupby('yrs_experience_field')['annual_sal_full_time'].mean().reset_index()

yrs_order = ['1 year or less', '2 - 4 years', '5-7 years', '8 - 10 years', '11 - 20 years','21 - 30 years','31 - 40 years','41 years or more']
df_yrs_avg['yrs_experience_field'] = pd.Categorical(df_yrs_avg['yrs_experience_field'], categories=yrs_order, ordered=True)

# Sort by the 'Month' column
sorted_df_yrs = df_yrs_avg.sort_values(by='yrs_experience_field')
sorted_df_yrs

fig = px.bar(sorted_df_yrs, x="yrs_experience_field", y="annual_sal_full_time", title='Salary by Years of Experience in Tech')
fig.update_layout(
    xaxis_title="Years of Experience in Field",
    yaxis_title="Average Annual Salary"
)
fig.show()

average_change = sorted_df_yrs['annual_sal_full_time'].diff().mean()
print(average_change)

/tmp/ipykernel_47584/4182547277.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



7109.801484230055


In [364]:

# Question 4: What percentage of respondents work remotely vs. in-office?
search_value = 'Remote'
total_countR = df_clean.isin([search_value]).sum().sum()
print(f"'{search_value}' exists in the DataFrame {total_countR} time(s).")

search_value = 'remote'
total_countr = df_clean.isin([search_value]).sum().sum()
print(f"'{search_value}' exists in the DataFrame {total_countr} time(s).")

search_value = 'WFH'
total_countWFH = df_clean.isin([search_value]).sum().sum()
print(f"'{search_value}' exists in the DataFrame {total_countWFH} time(s).")

total_count = total_countR + total_countr + total_countWFH
remotes=(total_count/len(df_clean)) *100
inoffices=100-remotes

print(f"So, approximately {remotes:,.2f}% work remotely, and {inoffices:,.2f}% work in office.")


'Remote' exists in the DataFrame 136 time(s).
'remote' exists in the DataFrame 12 time(s).
'WFH' exists in the DataFrame 5 time(s).
So, approximately 0.55% work remotely, and 99.45% work in office.


In [365]:
# Question 5: Which industry (besides tech) has the highest median salary?
df_clean['annual_sal_full_time'] = pd.to_numeric(df_clean['annual_sal_full_time'], errors='coerce')
df_industry = df_clean.groupby('industry')['annual_sal_full_time'].median().reset_index()
df_industry_sorted = df_industry.sort_values(by='annual_sal_full_time', ascending=False)

print('Industry besides tech with the highest median salary:',df_industry_sorted.iloc[0, 0])

Industry besides tech with the highest median salary: ESL Teacher


In [366]:
# Bonus Questions:
# Question 6: What's the salary gap between men and women in similar roles?
# Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?
# Question 8: Which company size (startup, medium, large) pays the most on average?

## Final Summary

**Summarize your findings here:**

1. **Median salary for Software Engineers in US:** $142,000
2. **Highest paying US state for tech:** Florida
3. **Salary increase per year of experience:** $7109.80 per year
4. **Remote vs office percentage:** 0.55% remote, 99.45% office
5. **Highest paying non-tech industry:** ESL Teacher

**Key insights:**
- Generally, as years of experience increase, salary increases (up until around 30 years of experience).
- It seems that the majority of workers reported working in-office
- ESL Teacher had the highest median salary outisde of tech

**Challenges faced:**
- When mapping the column names to new names print(repr(df_clean.columns)) helped me to get the proper title of each column so that it could be mappeed to a new name. 
- Certain colums representing numeric data were objects, so they had to be converted to numeric values

**What you learned about vibe coding:**
- Allows for more productivity, but you have to be careful about understanding what AI is suggesting
- When provided enough context, Cursor can create a pretty clear framework of actions you should take
- Allows you to focus more on outcomes than syntax
